# Evaluating the reproducibility of single-cell gene regulatory network inference algorithms

scNET evaluates Gene Regulatory Network inference algorithms based on reproducibility.
This markdown proposes a step by step tutorial for algorithm evaluation.

Datasets used for this tutorial are available in the `scNET_data` directory, or the link https://cloud.biologie.ens.fr/index.php/s/JuJgrIL1jC6yZh4/download, as described in the Github README. Data preprocessing was done as described in the article.

By default, this markdown performs reproducibility analysis for Human Retina data. The same analysis can be repeated on other biological contexts: CRC T-cells and 5 different types of hematopoietic cells by uncommenting its corresponding blocks of code in the `Loading pre-processed input scRNA-seq data` cells.

# Setting the scNET environment

Before loading data, make sure that the scNET environment has been correctly created and activated (see Github README). Additionally, run the following code to ensure an R interface with `jupyter notebook` 


In [23]:
###interface with R
%load_ext rpy2.ipython

Next, load R functions from the scNET repository 

In [ ]:
%%R

###load reproducibility functions and libraries
source('../scNET/Functions.R')

# Loading pre-processed input scRNA-seq data 

As described in the methods of the paper, raw count data corresponding to QC passed cells (as per the original articles) was first downloaded and pre-processed. Here, data will be loaded as both Python and R variables, according to each algorithm's specificity (GRNBoost2 and GENIE3 in Python, PPCOR in R).

In this cell, users can comment and uncomment lines depending on the biological context they'd like to analyze. The default is set to Human Retina data.

In [38]:
import glob

#RETINA
files= glob.glob('../scNET_data/RETINA/*.csv')

# #CRC T cells
# files= glob.glob('../scNET_data/CRC T-cells/*.csv')

# #Hematopoesis
# files= glob.glob('../scNET_data/HEMATO/*CLP.csv')

# files= glob.glob('../scNET_data/HEMATO/*DC.csv')

# files= glob.glob('../scNET_data/HEMATO/*Ery.csv')

# files= glob.glob('../scNET_data/HEMATO/*HSC.csv')

# files= glob.glob('../scNET_data/HEMATO/*Mono.csv')


Then, the following cell will load the selected data to Python. Additionally, a list of Transcription Factors (TF) to provide to the algorithms GRNBoost2 and GENIE3 will also be loaded (see GRNBoost2).

In [40]:
#Load data
import pandas as pd
from arboreto.utils import load_tf_names

First_dataset= pd.read_csv(files[0], sep=',', index_col=0)
First_exp= First_dataset.to_numpy()
Second_dataset= pd.read_csv(files[1], sep=',', index_col=0)
Second_exp= Second_dataset.to_numpy()
genes= First_dataset.columns

###Load TF list
tf_names= load_tf_names('../scNET_data/TF_names.txt')

IndexError: list index out of range

This next cell will load the same data into a R variable. Please make sure that the uncommented lines in this cell corresponds to the biological context selected in the first cell of this section. Again, the default is set to Human Retina data.

In [35]:
%%R

#RETINA
setwd('../scNET_data/RETINA/')
files= list.files(pattern='.csv')


#CRC T-cells
# setwd('../scNET_data/CRC T-cells')
# files= list.files(pattern='.csv')

# #Hematopoesis : CLP
# setwd('../scNET_data/HEMATO')
# files= list.files(pattern='CLP.csv')

# files= list.files(pattern='DC.csv')

# files= list.files(pattern='Ery.csv')

# files= list.files(pattern='HSC.csv')

# files= list.files(pattern='Mono.csv')


###load files 
data= lapply(files, read.csv, row.names=1)


Error in setwd("../scNET_data/RETINA/") : 
  작업디렉토리를 변경할 수 없습니다


# Running single-cell network inference algorithms

We will now apply GRNBoost2, GENIE3 and PPCOR to the loaded data.
To apply any additional algorithms, refer to Github README

## GRNBoost2

GRNBoost2 (Aibar, S., et al., ) takes as input an expression matrix and produces a list of TF to target interactions. This link list is formatted into 3 columns (column 1: TF, column2: Target, column3: link weight), and represents a directed network. A curated list of human transcription factors(Chawla, K., et al.) is provided as input to steer the inference. The code to apply GRNBoost2 to this data has been configured as described in https://arboreto.readthedocs.io/en/latest/. To speed up computation and produce networks for both datasets at once, a custom Dask client will help paralellize tasks.  

In practice, it is highly recommended to copy the following cell (preceded by the first two cells in the `Loading pre-processed input scRNA-seq data` section) into a seperate script and executing GRNBoost2 independantly, as the network inference is time consuming.

In [ ]:
### Applying GRNBoost2###
from arboreto.algo import grnboost2
from distributed import LocalCluster, Client

if __name__ == '__main__':
    # ex_matrix is a DataFrame with gene names as column names
    #build local cluster for parallel computation 
    local_cluster = LocalCluster(n_workers=4,
                                 threads_per_worker=1,
                                 memory_limit=10e9)
    custom_client = Client(local_cluster)
    
    #build networks 
    First_Network = grnboost2(expression_data=First_exp,
                        gene_names= genes,
                        tf_names= tf_names,
                        client_or_address=custom_client,
                        seed=100)
    Second_Network = grnboost2(expression_data=Second_exp,
                        gene_names= genes,
                        tf_names= tf_names,
                        client_or_address=custom_client,
                        seed=100)
    
    #close custom client
    custom_client.close()
    local_cluster.close()
    
    #export networks to results file
    
    First_Network.to_csv('../scNet/Results/GRNBoost2_Network1.tsv', sep='\t', index=False, header=False)
    Second_Network.to_csv('../scNET/Results/GRNBoost2_Network2.tsv', sep='\t', index=False, header=False)

## GENIE3

GENIE3 (Aibar, S., et al.) was also provided with a TF list, and produces a link list in the same format as GRNBoost2. Similarly, the code to apply GENIE3 has been configured as described in https://arboreto.readthedocs.io/en/latest/, and the inference is parallelized.  

Again,it is highly recommended to copy the following cell (preceded by the first two cells in the `Loading pre-processed input scRNA-seq data` section) into a seperate script. This is especially pertinent because GENIE3 is  computationally heavier than GRNBoost2

In [ ]:
###Applying GENIE3###
from arboreto.algo import genie3
from distributed import LocalCluster, Client

if __name__ == '__main__':
    # ex_matrix is a DataFrame with gene names as column names
    #build local cluster for parallel computation 
    local_cluster = LocalCluster(n_workers=4,
                                 threads_per_worker=1,
                                 memory_limit=10e9)
    custom_client = Client(local_cluster)
    
    #build networks 
    First_Network = genie3(expression_data=First_exp,
                        gene_names= genes,
                        tf_names= tf_names,
                        client_or_address=custom_client,
                        seed=100)
    Second_Network = genie3(expression_data=Second_exp,
                        gene_names= genes,
                        tf_names= tf_names,
                        client_or_address=custom_client,
                        seed=100)
    
    #close custom client
    custom_client.close()
    local_cluster.close()
    
    #export networks to results file
    First_Network.to_csv('./scNet/Results/GENIE3_Network1.tsv', sep='\t', index=False, header=False)
    Second_Network.to_csv('./scNET/Results/GENIE3_Network2.tsv', sep='\t', index=False, header=False)

## PPCOR

PPCOR calculates partial Spearman correlation coefficients between ALL possible gene pairs based on gene expression values. This algorithm produces an adjacency matrix, with genes in both columns and rows, and whose values correspond to correlation coefficients. A post-processing step is thus necessary for PPCOR results to 1) select interactions that are the most significant and 2) reformat links into a list that resembles GRNBoost2 and GENIE3 outputs. Note that PPCOR produces an undirected network after post-processing.

Links are selected by setting a threshold on correlation coefficient values. The threshold is determined by calculating a significant correlation coefficent value, given a statistical power P=0.8, a significance level alpha=0.05, and N= the number of cells in the corresponding dataset.

It was noted during the analysis that PPCOR can give non-valid correlation coefficient values (>1 and <-1). The post-processing function checks for the percentage of valid links in ppcor results. If coefficients are not in the expected range, no networks will be produced. For details on the post-processing code, see `Functions.R`

In [33]:
%%R

#Applying PPCOR 
library(ppcor)
PPCOR.res.net1= pcor(data[[1]], method= 'spearman')
PPCOR.res.net2= pcor(data[[2]], method= 'spearman')
ppcor.res= list(net1= PPCOR.res.net1, net2=PPCOR.res.net2)

setwd('../scNET/Results/')
saveRDS(ppcor.res, file='PPCOR_raw_results.Rds')

#calculate threhsold according to ncells in each dataset
ncells1= nrow(data[[1]])#cells in rows
thresh1=pwr.r.test(n = ncells1, sig.level = 0.05, power = 0.8, alternative = "two.sided")$r
ncells2= nrow(data[[2]])#cells in rows
thresh2=pwr.r.test(n = ncells2, sig.level = 0.05, power = 0.8, alternative = "two.sided")$r

#Post-process PPCOR results to produce networks
ppcor.net1=ppcor.post(ppcor.res[[1]], coefficient.threshold = thresh1)
ppcor.net2=ppcor.post(ppcor.res[[2]], coefficient.threshold = thresh2)

#Export PPCOR results
if(nrow(ppcor.net1)!= 0 & nrow(ppcor.net2)!=0){
    write.csv(ppcor.net1, 'PPCOR_Network1.tsv', quote=F, sep='\t')
    write.csv(ppcor.net2, 'PPCOR_Network2.tsv', quote=F, sep='\t')
}else{message('Networks produced by PPCOR can not be analysed for reproducibility')}

[1] "PPCOR_ RETINA _res.Rds"


## GeneNet

GeneNet first converts a correlation network into a partial correlation graph through the R function 'ggm.estimate.pcor()'. Subsequently, a partial ordering of the nodes is established by multiple testing of the log-ratio of standardized partial variances with functions 'network.test.edges()' and 'extract.network()'.


In [ ]:
%%R

#Applying GeneNet
library(GeneNet)
data1.pc = ggm.estimate.pcor(data[[1]])
data1.edges = network.test.edges(data1.pc, direct=TRUE, fdr=TRUE)
data1.net = extract.network(data1.edges )

data2.pc= ggm.estimate.pcor(data[[2]])
data2.edges = network.test.edges(data2.pc, direct=TRUE, fdr=TRUE)
data2.net = extract.network(data2.edges)

#renaming links
nodes= colnames(data[[1]])
transfo1= transform.node.labels(data1.net, nodes)
transfo2= transform.node.labels(data2.net, nodes)

#reassign weights
GeneNet.res1= cbind(transfo1, data1.net[,1])
GeneNet.res2= cbind(transfo2, data2.net[,1])

#Export result files to 
write.csv(GeneNet.res1,'GeneNet_Network1.tsv', quote=F, sep='\t')
write.csv(GeneNet.res2,'GeneNet_Network2.tsv', quote=F, sep='\t')


## CLR

The Context Likelihood of Relatedness (CLR) approach first computes mutual information value between all possible gene pairs. Then the algorithm adjusts the link weights by using the background distribution of mutual information (MI) values as a reference. This allows the algorithm to filter out false postive gene links.

This algorithm can be used by loading the R package minet (Mutual Information NETworks). The 'build.mim' function takes as input an expression matrix purged of all zero counts and computes a mutual information matrix (MIM), an adjacency matrix of all mutual information values. The 'clr' function takes this MIM as input and completes the inference process.

However, since the data has not yet been purged of zero counts, the function 'remove.zero.counts' included in  functions.R will remove any genes that have a zero count in any cells from the original expression matrix before computing mutual information.


In [ ]:
%%R

#load minet library
library(minet)

#remove genes with zero counts
new1= remove.zero.count(data[[1]])
new2= remove.zero.count(data[[2]])

##check number of removed genes
ncol(net1)- ncol(new1)
ncol(net2)- ncol(new2)

##Apply CLR
net1.mim= build.mim(new1, estimator = 'spearman', disc = 'none')
net2.mim= build.mim(new2, estimator = 'spearman', disc = 'none')

net1.clr= clr(net1.mim)
net2.clr= clr(net2.mim)

#Save result networks
clr1=BuildNetworks(net1.clr)
clr2=BuildNetworks(net2.clr)

write.csv(clr1,'CLR_Network1.tsv', quote=F, sep='\t')
write.csv(clr2,'CLR_Network2.tsv', quote=F, sep='\t')

## Optional: Custom algorithm comparison

It is possible to test any other single cell gene network inference algorithms for reproducibility using the measures that we have developped. Just make sure to use two independant expression matrices as input, as shown above. Also, the networks inferred with these algorithms must be formatted as a dataframe with 3 columns (column 1 = gene 1, column 2= gene 2, and column 3= link weights). It is also important to check if the network is directed or not- whether or not the gene to gene interaction has an explicit direction (gene 1 regulates gene 2, for example)- and adjust the 'Directed' variable accordingly when proceeding with this notebook.

## Filtering networks

Due to the various methods used to infer networks, the obtained networks can vary greatly in size, from a few thousand links to a few million. This is why we chose to determine a maximum number of links to be analyzed, called k. If the number of links to analyze in a certain network is lower than k, the full network will be used.

In [ ]:
%%R

#determine k
k= 100000

#select block of code corresponding to the networks to load. Comment out all others
setwd('../scNET/Results/')

net1= read.table('GRNBoost2_Network1.tsv', quote=F)
net2= read.table('GRNBoost2_Network2.tsv', quote=F)

# net1= read.table('GENIE3_Network1.tsv', quote=F)
# net2= read.table('GENIE3_Network2.tsv', quote=F)

# net1= read.table('PPCOR_Network1.tsv', quote=F)
# net2= read.table('PPCOR_Network2.tsv', quote=F)

# net1= read.table('GeneNet_Network1.tsv', quote=F)
# net2= read.table('GeneNet_Network2.tsv', quote=F)

# net1= read.table('CLR_Network1.tsv', quote=F)
# net2= read.table('CLR_Network2.tsv', quote=F)

In [ ]:
%%R

#

# Algorithm reproducibility evaluation

Once all algorithms have been applied on both datasets, their reproducibility can be evaluated. Three metrics are calculated: Intersection index, Weighted Jaccard Similarity (WJS) and RcisTarget score (see Methods for more details, and `Functions.R` for details on the code). 

Before calculating the RcisTarget score, the RcisTarget algorithm(Aibar, S., et al.) must first be applied to all networks. Note that here, RcisTarget has been customized to select only High Confidence links, that are supported by strong motif evidence (see Methods)

In [ ]:
%%R

#execute Rcistarget
setwd('../scNET/')

Rcis_GRNBoost2 = Custom.Rcis(input.dir='scNET/Results',
                     pattern='GRNBoost2',
                     chosenDb="hg19-tss-centered-5kb-7species.mc9nr.feather",
                     output.dir='scNET/Results',
                     MinGenesetSize=0, 
                     directed=T)

Rcis_GENIE3 = Custom.Rcis(input.dir='scNET/Results',
                     pattern='GENIE3',
                     chosenDb="hg19-tss-centered-5kb-7species.mc9nr.feather",
                     output.dir='scNET/Results',
                     MinGenesetSize=0, 
                     directed=T)

Rcis_PPCOR = Custom.Rcis(input.dir='scNET/Results',
                     pattern='PPCOR',
                     chosenDb="hg19-tss-centered-5kb-7species.mc9nr.feather",
                     output.dir='scNET/Results',
                     MinGenesetSize=0, 
                     directed=F)


Once RcisTarget has been applied, we can now calculate reproducibility scores for each algorithm.

In [ ]:
%%R

###Results
Results= Reproducibility.stats(c('GRNBoost2*.tsv','GENIE3*.tsv', 'PPCOR*.tsv'),
                               Results.dir= 'scNET/Results')

Results[,'RcisTarget_index']= c(Rcis.percent(Rcis_GRNBoost2),
                               Rcis.percent(Rcis_GENIE3),
                               Rcis.percent(Rcis_PPCOR))

Results

The above results can be visualized as barplots, as follows:

In [ ]:
%%R

###Plotting results

ggplot(data=Results, aes(x=Algorithm, y= Intersection_index, fill='')) +
geom_bar(stat="identity")+
ggtitle('Intersection index comparison')

ggplot(data=Results, aes(x=Algorithm, y= WJS, fill='')) +
geom_bar(stat="identity")+
ggtitle('WJS comparison')

ggplot(data=Results, aes(x=Algorithm, y= RcisTarget_index, fill='')) +
geom_bar(stat="identity")+
ggtitle('RcisTarget score comparison')

# Quantile cutting

In this part of the analysis, we threshold network weights according to certain percentiles (40, 80 and 90%). Calculating reproducibility measures for each percentile allows us to compare algorithm stability (see Methods)

In [ ]:
%%R

###Quantile Cutting
GRN= lapply(list.files(pattern='GRNBoost2*.tsv'), read.table)
GEN= lapply(list.files(pattern='GENIE3*.tsv'), read.table)
PPCOR= lapply(list.files(pattern='PPCOR*.tsv'), read.table)

thresh.GRN=quantile.stats(GRN[[1]], GRN[[2]], c(0.4, 0.8, 0.9), Directed=T, label='GRNBoost2')
thresh.GEN=quantile.stats(GEN[[1]], GEN[[2]], c(0.4, 0.8, 0.9), Directed=T, label='GENIE3')
thresh.PPCOR=quantile.stats(PPCOR[[1]], PPCOR[[2]], c(0.4, 0.8, 0.9), Directed=F, label='PPCOR')

all.quantiles= rbind(thresh.GRN, thresh.GEN)
all.quantiles= rbind(all.quantiles, thresh.PPCOR)

all.quantiles

The above results can be visualized, as follows:

In [ ]:
%%R

###Plots
ggplot(data=all.quantiles, aes(x=Quantile, y= intersection, group=Algorithm)) +
  geom_line(aes(color=Algorithm))+
  geom_point(aes(color=Algorithm))+
  ggtitle('Intersection Index comparison according to Quantiles')

ggplot(data=all.quantiles, aes(x=Quantile, y= WJS, group=Algorithm)) +
  geom_line(aes(color=Algorithm))+
  geom_point(aes(color=Algorithm))+
  ggtitle('WJS comparison according to Quantiles')